In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from tf_lab.fundamentals.utils import set_visible_GPUs, reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


import tf_lab.autopredictors.scripts.virt_scan_data as vscan

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, points_extension


from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch
        
from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs, accuracy_of_completion, \
                                           coverage_of_completion, save_reconstructions
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [11]:
GPU = 2
exp_counter = '9'
loss = 'emd'
do_training = True
load_model_conf = False
do_evaluation = True

incomplete_n_samples = 2048
complete_n_samples = 4096
val_percent = .10
dropout_keep_prob = .8
seed = 42

experiment_name = exp_counter + '_all_classes_' + str(incomplete_n_samples) + '_' \
                  + str(complete_n_samples) + 'pts_' + loss

top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'

n_input = [incomplete_n_samples, 3]
n_output = [complete_n_samples, 3] 

train_dir = osp.join(top_data_dir, 'OUT/models/incomplete_pclouds/paper_vanilla_vscan')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

if loss == 'emd':
    max_training_epochs = 100
elif loss == 'chamfer':
    max_training_epochs = 300
    
max_evaluation_epochs = max_training_epochs

In [4]:
# Load Data of All Classes.
class_to_syn_id = shape_net_category_to_synth_id()
all_classes = vscan.all_classes
n_threads = 50

first = class_to_syn_id[all_classes[0]]
train_data, val_data, test_data = vscan.load_train_val_test_vscan_paper(first, n_threads,\
                                                                        complete_n_samples=complete_n_samples,\
                                                                        incomplete_n_samples=incomplete_n_samples,
                                                                        val_percent=val_percent)
for model_class in vscan.all_classes[1:]:
    class_syn_id = class_to_syn_id[model_class]
    curr_train, curr_val, curr_test = vscan.load_train_val_test_vscan_paper(class_syn_id, n_threads,\
                                                              complete_n_samples=complete_n_samples,\
                                                              incomplete_n_samples=incomplete_n_samples,\
                                                              val_percent=val_percent)
    train_data.merge(curr_train)
    test_data.merge(curr_test)
    val_data.merge(curr_val)

train_data.shuffle_data();

4045 files containing complete point clouds were found.
19800 incomplete point clouds were loaded.
4470 incomplete point clouds were loaded.
1572 files containing complete point clouds were found.
7800 incomplete point clouds were loaded.
1632 incomplete point clouds were loaded.
7497 files containing complete point clouds were found.
29640 incomplete point clouds were loaded.
5922 incomplete point clouds were loaded.
6778 files containing complete point clouds were found.
30000 incomplete point clouds were loaded.
6000 incomplete point clouds were loaded.
2318 files containing complete point clouds were found.
11100 incomplete point clouds were loaded.
2808 incomplete point clouds were loaded.
3173 files containing complete point clouds were found.
15600 incomplete point clouds were loaded.
3438 incomplete point clouds were loaded.
8509 files containing complete point clouds were found.
30000 incomplete point clouds were loaded.
6000 incomplete point clouds were loaded.
1939 files con

In [5]:
print train_data.num_examples + val_data.num_examples
print test_data.num_examples

153540
32304


In [33]:
# # Verification we didn't mix train-test-val data.
# tr = train_data.full_epoch_data(shuffle=False)
# va = val_data.full_epoch_data(shuffle=False)
# te = test_data.full_epoch_data(shuffle=False)

# train_set = set([i[:-6] for i in tr[1]])
# val_set = set([i[:-6] for i in va[1]])
# test_set = set([i[:-6] for i in te[1]])

# c1 = len(test_set.intersection(train_set)) == 0
# c2 = len(test_set.intersection(val_set)) == 0
# c3 = len(train_set.intersection(val_set)) == 0

# assert(c1 and c2 and c3)

# pp = test_data.next_batch(1)
# pinc = pp[2].reshape(incomplete_n_samples, 3)
# pcom = pp[0].reshape(complete_n_samples, 3)
        
# score1 = accuracy_of_completion(pinc, pcom, 0.02, ret_dists=False)
# print score1
# score2, c2 = coverage_of_completion(pcom, pinc, 0.02, ret_dists=True)

# Point_Cloud(points=pinc).plot();
# Point_Cloud(points=pcom).plot(c=c2);
# print pp[1]

In [9]:
tf.set_random_seed(seed)

if load_model_conf:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
else:
    decoder_args = {'layer_sizes': [1024, np.prod(n_output)],
                    'non_linearity': tf.nn.relu
                   }

    encoder_args = {'dropout_prob': dropout_keep_prob}
    
    conf = Conf(
                n_input = n_input,
                n_output = n_output,
                denoising = True,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 1,
                learning_rate = 0.0005,
                encoder = enc_dec.encoder_with_convs_and_symmetry,
                encoder_args = encoder_args,
                decoder = enc_dec.decoder_with_fc_only,
                decoder_args = decoder_args
               )
    
    conf.allow_gpu_growth = False
    conf.consistent_io = False
    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))
    
reset_tf_graph()
set_visible_GPUs([GPU])
ae = PointNetAutoEncoder(experiment_name, conf)

In [ ]:
ae.restore_model(conf.train_dir, 4)
training_stats = []
if do_training:
    training_stats.append(ae.train(train_data, conf))

('Epoch:', '0005', 'training time (minutes)=', '26.2482', 'loss=', '230.829998694')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/9_all_classes_2048_4096pts_emd/models.ckpt-5 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0006', 'training time (minutes)=', '26.1660', 'loss=', '226.670274832')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/9_all_classes_2048_4096pts_emd/models.ckpt-6 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0007', 'training time (minutes)=', '26.1540', 'loss=', '223.677349669')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/9_all_classes_2048_4096pts_emd/models.ckpt-7 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0008', 'training time (minutes)=', '25.8984', 'loss=', '220.757881996')
INFO:tensorflow:/orion

In [ ]:
if do_evaluation:
    # Pick the epoch that minimizes the loss on the validation dataset.
    saved_epochs = np.array(read_saved_epochs(conf.train_dir))
    allowable_epochs = saved_epochs[saved_epochs <= max_evaluation_epochs]
    val_stats = eval_model(ae, conf, val_data, epochs=allowable_epochs, verbose=True)
    val_loss = np.min(val_stats[:,1])
    best_epoch = int(val_stats[np.argmin(val_stats[:,1]), 0])
    print 'Best epoch = %d.' % (best_epoch,) 
        
    ae.restore_model(conf.train_dir, best_epoch)
    top_save_dir = osp.join(conf.train_dir, 'output', 'epoch_' + str(best_epoch))
    save_dir = osp.join(top_save_dir, 'test_predictions')
    test_recon, test_loss, test_feed, test_ids, test_gt = ae.evaluate(test_data, conf)
    save_reconstructions(save_dir, test_recon, test_gt, test_feed, test_ids) # save ply files of test data.    
    train_loss = ae.evaluate(train_data, conf)[1]
    
    # Report Accuracy and Coverage of test data.
    n_examples = len(test_recon)
    pred_scores = np.zeros((n_examples, 2))
    for i in xrange(n_examples):
        gt = test_gt[i]
        pred = test_recon[i]    
        pred_scores[i, 0] = accuracy_of_completion(pred, gt, thres=0.02, ret_dists=False)
        pred_scores[i, 1] = coverage_of_completion(gt, pred, thres=0.02, ret_dists=False)
    
    print 'Test Median-Accuracy-Coverage:', np.median(pred_scores[:, 0]), np.median(pred_scores[:, 1])
    
    with open(osp.join(top_save_dir, 'stats.txt'), 'w') as fout:
        fout.write('Best Validation Epoch = %d\n' % (best_epoch))
        fout.write('Validation loss = %f\n' % (val_loss))
        fout.write('Train loss = %f\n' % (train_loss))
        fout.write('Test loss = %f\n' % (test_loss))
        fout.write('Gen. Error (abs, per) = %f %f\n' % (abs(test_loss-train_loss),  abs(test_loss-train_loss) / train_loss ))
        fout.write('Test Median-Accuracy-Coverage = %f %f\n' % (np.median(pred_scores[:, 0]), np.median(pred_scores[:, 1])))